# Import packages

In [1]:
import torch
import torch.nn as nn
import torchdiffeq
from torchdiffeq import odeint
from xitorch.interpolate import Interp1D
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt

In [3]:
# Generate x and y in y = log(x)
log_V_range = [-15, 2]
N_seq = 10000
y = torch.rand([N_seq, 1]) * (log_V_range[1] - log_V_range[0]) + log_V_range[0]
x = torch.pow(10., y)

# Define the NN to learn log function

In [4]:
class PP(nn.Module):
    # Constructor
    def __init__(self, NNs, input_dim = 1, output_dim = 1):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, NNs[0]), 
            nn.ReLU(),
        )
        
        for i in range(len(NNs) - 1):
            self.fc.append(nn.Linear(NNs[i], NNs[i + 1]))
            self.fc.append(nn.ReLU)
        
        self.fc.append(nn.Linear(NNs[-1], output_dim))
    
    # Forward function
    def forward(self, x):
        return self.fc(x)
        
        

# Training for $y = \log(x)$